### Function that takes iterates over all files and produces a concatenated dataframe
#### Data produced will be stored in DataFrame named as 'finalDataFrame' that will persist across Notebooks

In [65]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import time # workaround to get past the issue of Google Sheets API timing out

In [66]:
#retrieve filesInDriveDict
%store -r filesInDriveDict

# Configure the connection 
scopes = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive.readonly']
key_file_location = 'gsheetsprivkey\serviceaccount.json'

# Give the path to the Service Account Credential json file 
credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_location,scopes)

# Authorise your Jupyter Notebook to connect to Google Drive API using private key credentials in 'credentials'
gc = gspread.authorize(credentials)

In [67]:
#List comprehension to pull up all the ids from the filesInDriveDict
fileKeyIDs = [val['id'] for key,val in filesInDriveDict.items()]

#initialize a list with one value , viz number of files in fileKeyIDs
fileIDChunkList = [len(fileKeyIDs)]

# breaking 27 into [7,10,10]
while fileIDChunkList[0] > 10:
    fileIDChunkList.append(10)
    fileIDChunkList[0] = fileIDChunkList[0]-10

In [68]:
#Helper functions

# Note: this function is applied in context of each season
# Add points for each team before the game starts
def addPointsBeforeGame(pd_data):
    #unique teams
    uniqueTeams = pd_data['HomeTeam'].unique()

    #build dict with all teams in that season; initialize to zero for each
    teamPoints = {uniqueTeams[i]:0 for i in range(0,len(uniqueTeams))};
    
    # Adding new feature: Points before each game starts
    # PBGH = PointsBeforeGameHomeTeam in that season
    # PBGA = PointsBeforeGameAwaayTeam in that season
    pd_data['PBGH'] = 0
    pd_data['PBGA'] = 0

    for index,row in pd_data.iterrows():
        #set points before start of game into DataFrame
        pd_data.loc[index,'PBGH'] = teamPoints[row['HomeTeam']]
        pd_data.loc[index,'PBGA'] = teamPoints[row['AwayTeam']]
        #check who won and who lost
        if (row['FTR']=='D'):
            teamPoints[row['HomeTeam']] += 1
            teamPoints[row['AwayTeam']] += 1
        elif (row['FTR']=='H'):
            teamPoints[row['HomeTeam']] += 3
        elif (row['FTR']=='A'):
            teamPoints[row['AwayTeam']] += 3
    return 

def getWinLossDrawStreak(finalData):
    
    #across all seasons thus far
    uniqueTeams = finalData['HomeTeam'].unique()
    
    #build dict with ALL teams; initialize to ----- for each
    teamStreak = {uniqueTeams[i]:'WWWWW' for i in range(0,len(uniqueTeams))}; 
    
    print(teamStreak['Man United'])
    # Adding new feature: Win streak before game
    # HomeStreak = Streak of home team before the game
    # AwayStreak = Streak of away team before the game
    # initialize to random initial value
    finalData['HomeStreak'] = 'W'
    finalData['AwayStreak'] = 'L'
    
    finalData['HomeStreak'].fillna('', inplace=True)
    finalData['AwayStreak'].fillna('', inplace=True)
    
    for index,row in finalData.iterrows():
        
        #Populate streak data into columns before game outcome is counted
        finalData.loc[row,'HomeStreak'] = teamStreak[row['HomeTeam']];
        finalData.loc[row,'AwayStreak'] = teamStreak[row['AwayTeam']];
        
        #check who won and who lost
        if (row['FTR']=='D'):
            teamStreak[row['HomeTeam']] = 'D' + teamStreak[row['HomeTeam']]
            teamStreak[row['AwayTeam']] = 'D' + teamStreak[row['AwayTeam']]
        elif (row['FTR']=='H'):
            teamStreak[row['HomeTeam']] = 'W' + teamStreak[row['HomeTeam']]
            teamStreak[row['AwayTeam']] = 'L' + teamStreak[row['AwayTeam']]
        elif (row['FTR']=='A'):
            teamStreak[row['HomeTeam']] = 'L' + teamStreak[row['HomeTeam']]
            teamStreak[row['AwayTeam']] = 'W' + teamStreak[row['AwayTeam']]
        
        #snipping off last character since above if block added a result to start
        teamStreak[row['HomeTeam']] = teamStreak[row['HomeTeam']][0:-1]
        teamStreak[row['AwayTeam']] = teamStreak[row['AwayTeam']][0:-1]
    return



In [15]:
#initialize new 
finalDataFrame = pd.DataFrame()

#variables used to split fileKeyIDs
start = end = 0

# Expected to take over 100 seconds per 10 files due to time.sleep(). Expect 30 files to take 300 seconds (i.e. 5 mins)
for item in fileIDChunkList:
    end = start + item
    for keyID in fileKeyIDs[start:end]:
        #spreadsheet_key is the internal key stored in Google drive for that file
        tempWorkbook = gc.open_by_key(keyID)
        #By default load 1st sheet of the workbook , i.e. index = 0
        tempSheet = tempWorkbook.get_worksheet(0)
        #get_all_values returns list of lists with first list as column headers
        tempValues = tempSheet.get_all_values()
        # Pulling the data and transform it to the data frame .1st row is header , remaining are actual values
        temp_pd_data = pd.DataFrame(tempValues[1:], columns = tempValues[0])
        #converting string format dates to datetime object
        temp_pd_data['Date'] = pd.to_datetime(temp_pd_data['Date'])
        addPointsBeforeGame(temp_pd_data)
        #iteratively concatenating
        finalDataFrame = pd.concat([finalDataFrame,temp_pd_data])
    start += item;
    #if not on the last item (since 'end' is equal to length of fileKeyIDs on last step), proceed to pause for 100 secs
    if end < len(fileKeyIDs):
        time.sleep(100)

In [17]:
%store finalDataFrame

Stored 'finalDataFrame' (DataFrame)


In [69]:
finalDataFrame

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHW,HO,AO,HBP,ABP,SYH,SYD,SYA,HomeStreak,AwayStreak
0,E0,2019-09-08,20:00,Liverpool,Norwich,4,1,H,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
1,E0,2019-10-08,12:30,West Ham,Man City,0,5,A,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
2,E0,2019-10-08,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
3,E0,2019-10-08,15:00,Burnley,Southampton,3,0,H,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
4,E0,2019-10-08,15:00,Crystal Palace,Everton,0,0,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,E0,1994-07-05,NaN,Sheffield Weds,Man City,1,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
458,E0,1994-07-05,NaN,Swindon,Leeds,0,5,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
459,E0,1994-07-05,NaN,Tottenham,QPR,1,2,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
460,E0,1994-07-05,NaN,West Ham,Southampton,3,3,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L


In [70]:
temp_finalData = finalDataFrame
temp_finalData

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHW,HO,AO,HBP,ABP,SYH,SYD,SYA,HomeStreak,AwayStreak
0,E0,2019-09-08,20:00,Liverpool,Norwich,4,1,H,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
1,E0,2019-10-08,12:30,West Ham,Man City,0,5,A,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
2,E0,2019-10-08,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
3,E0,2019-10-08,15:00,Burnley,Southampton,3,0,H,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
4,E0,2019-10-08,15:00,Crystal Palace,Everton,0,0,D,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,E0,1994-07-05,NaN,Sheffield Weds,Man City,1,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
458,E0,1994-07-05,NaN,Swindon,Leeds,0,5,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
459,E0,1994-07-05,NaN,Tottenham,QPR,1,2,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
460,E0,1994-07-05,NaN,West Ham,Southampton,3,3,D,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W,L
